MBART(encoder-decoder model)




1.   Input Representation (Embeddings)
Suppose we have a source sentence in English.
Tokenization:
Tokenizer converts it to discrete tokens
Convert tokens → IDs
Embedding Layer:Each token ID is mapped to a vector via learned embedding matrix
𝐸
∈
𝑅
𝑉
×
𝑑, where
𝑉
= vocab size,
𝑑
 = embedding dimension:


1.   Positional Encoding
Transformers are sequence-agnostic, so we add positional information

1.   Encoder:Multi-head self attention
The encoder processes all tokens simultaneously using self-attention
Multi-head attention:
Split embeddings into
h heads, compute attention in parallel
Concatenate heads and project
This allows the model to capture different types of relationships between words.

1.   Encoder Feed-Forward Layer:After attention, each token passes through a position-wise feed-forward network to capture non-linearities bewteen the words.
2.   Decoder: Masked Self-Attention + Cross-Attention:
The decoder predicts target tokens:
Masked Self-Attention:
Only attends to previous target tokens;
Cross-Attention:
Decoder attends to encoder outputs.
Allows decoder to align source and target tokens.


2.   Output Probability Distribution
The decoder produces hidden vector
ℎ
𝑡
for token
𝑡

Pass through linear + softmax to get probabilities over vocab:

It is different than normal BART cus it has special language tokens which makes it compatible with multilingual languages






In [1]:
!pip install transformers datasets sacrebleu sentencepiece accelerate


In [2]:
from datasets import load_dataset
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    set_seed,
)

In [3]:
import numpy as np
import sacrebleu
import torch

In [4]:
# -------- Config --------
MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"
SRC_LANG = "en_XX"
TGT_LANG = "hi_IN"
MAX_SOURCE_LENGTH = 128
MAX_TARGET_LENGTH = 128
EPOCHS = 2           # increase for better results
BATCH = 4
LR = 5e-5
OUTPUT_DIR = "./mbart-en-hi"
# ------------------------

In [5]:
set_seed(42)

In [6]:
print("Loading IIT Bombay English-Hindi dataset...")
dataset = load_dataset("cfilt/iitb-english-hindi")
train_ds = dataset["train"].select(range(1000))      # subset for quick demo
val_ds = dataset["validation"].select(range(200))

Loading IIT Bombay English-Hindi dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
train_ds[0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [8]:
!pip install bitsandbytes accelerate

In [9]:
from transformers import AutoModelForSeq2SeqLM

In [10]:
print("Loading model and tokenizer...")
tokenizer = MBart50TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/mbart-large-50-many-to-many-mmt",
    load_in_8bit=True,          # very memory-efficient
    device_map="auto"
)
tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG

Loading model and tokenizer...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [11]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

In [12]:
model = get_peft_model(model, config)

In [13]:
def preprocess(batch):
    # Extract list of English and Hindi sentences
    en_texts = [item["en"] for item in batch["translation"]]
    hi_texts = [item["hi"] for item in batch["translation"]]

    # Tokenize source (English)
    inputs = tokenizer(
        en_texts,
        max_length=MAX_SOURCE_LENGTH,
        truncation=True,
        padding="max_length",
    )

    # Tokenize target (Hindi)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            hi_texts,
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding="max_length",
        )

    # Replace pad token ids with -100 (ignored by loss)
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    inputs["labels"] = labels["input_ids"]
    return inputs


In [14]:
print("Tokenizing datasets...")
train_tokenized = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
val_tokenized = val_ds.map(preprocess, batched=True, remove_columns=val_ds.column_names)


Tokenizing datasets...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
def postprocess(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [[l.strip()] for l in labels]
    return preds, labels

In [17]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    preds, refs = postprocess(decoded_preds, decoded_labels)
    bleu = sacrebleu.corpus_bleu(preds, refs)
    return {"bleu": bleu.score}


In [18]:
model.config.forced_bos_token_id = tokenizer.lang_code_to_id[TGT_LANG]

In [19]:
args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-5,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    save_total_limit=1,
    logging_steps=50,

)


In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-1575370792.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [21]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [22]:
print("Training started...")
trainer.train()
print("Training complete. Saving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


Training started...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


...

/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,1.723700
100,1.697200
150,1.567000
200,1.337400
250,1.383400
300,1.496500
350,1.364100
400,1.438000
450,1.448900
500,1.084700


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Training complete. Saving model...


('./mbart-en-hi/tokenizer_config.json',
 './mbart-en-hi/special_tokens_map.json',
 './mbart-en-hi/sentencepiece.bpe.model',
 './mbart-en-hi/added_tokens.json',
 './mbart-en-hi/tokenizer.json')

In [24]:
model.to("cuda" if torch.cuda.is_available() else "cpu")
examples = [
    "Good morning! How are you?",
    "I love learning new languages.",
    "Where is the nearest railway station?",
]
tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG
for text in examples:
    # Tokenize and move input to device
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    # Generate output tokens
    output_tokens = model.generate(
        **inputs,
        num_beams=4,
        max_length=64,
        forced_bos_token_id=tokenizer.lang_code_to_id[TGT_LANG]  # ensures correct target language
    )

    # Decode tokens to text
    translated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    print(f"Input: {text}")
    print(f"Translation: {translated_text}")
    print("-" * 50)


Input: Good morning! How are you?
Translation: सुथरी! आप कैसे हैं?
--------------------------------------------------
Input: I love learning new languages.
Translation: मैं नई भाषाओं को सीखना पसंद करता हूं।
--------------------------------------------------
Input: Where is the nearest railway station?
Translation: निकटतम रेलवे स्टेशन कहां है?
--------------------------------------------------
